In [1]:
import astropy.io.fits as fits
import numpy as np

In [2]:
# Open catalogs
star_index_catalog = fits.open('/global/cfs/cdirs/lsst/shared/external/roman/v0.1_static/truth/dc2_test_index_star.fits')[1].data
observing_sequence = fits.open('/global/cfs/cdirs/lsst/shared/external/roman/v0.1_static/truth/observing_sequence_hlsonly_5yr.fits')[1].data

In [4]:
### Build new array
# Initialize new array. 7 columns, defined here. 
columns = [('ind',int), ('ra',float), ('dec',float), ('mag_1',float), ('mag_2',float), ('mag_3',float), ('mag_4',float)]
array = np.zeros((len(star_index_catalog), len(columns)))

# Fill in inds, ras, decs. 
array[:, 0] = star_index_catalog['ind']
array[:, 1] = star_index_catalog['ra'] # Still radians
array[:, 2] = star_index_catalog['dec'] # Still radians

In [5]:
### Grab filters from observing_sequence based on dither in star_index_catalog, then fill in mags in those filters
dithers = star_index_catalog['dither']
observing_sequence_reordered = observing_sequence[dithers]
# observing_sequence_reordered matches 1-to-1 with star_index_catalog
filters = observing_sequence_reordered['filter']

# Fill mag_i into array for each filter 1-4:
for i in [1, 2, 3, 4]:
    # This line fills mag_i with whatever is in star_index_catalog['mag'] if filter==i on that line, otherwise it fills in 0.
    mag_i = np.where(filters==i, star_index_catalog['mag'], 0)
    array[:, i+2] = mag_i # i+2 is because in columns, mag_1 is at index 3, mag_2 at index 4, so on. 

In [53]:
%%time
### Condense observations of the same star in different filters into one line. 
# Get unique inds, as well as their locations in array
unique_inds, indexes = np.unique(array[:, 0], return_index=True)

# Instead of modifying unique_array, build new one of the correct length, one line per unique star. 
final_array = np.zeros((len(unique_inds), len(columns)))
final_array[:, 0] = unique_inds
# Pull ras and decs from array using indexes of unique_inds
final_array[:, 1] = np.degrees(array[indexes, 1])
final_array[:, 2] = np.degrees(array[indexes, 2])

j = -1 # Index into final_array. Labels the line we're currently modifiying. 
ind_ref = None # Which ind we're currently pulling mags for. 
# Loop through lines of array, and based on ind, fill in the mags of the appropriate line of final_array if they haven't 
# alreayd been filled in
for line in array:
    ind = line[0]
    if ind != ind_ref:
        j += 1
        ind_ref = ind
    
    for filter in [1, 2, 3, 4]:
        if final_array[j][filter+2] <= 0 and line[filter+2] > 0:
            final_array[j][filter+2] = line[filter+2]

final_array = np.core.records.fromarrays(final_array.T, dtype=columns)

CPU times: user 1.35 s, sys: 16.1 ms, total: 1.37 s
Wall time: 1.37 s
